# Graphs

Uma rede neural é um grafo de operações matemáticas. Os nós em cada camada (exceto a camada de entrada) realiza uma operação matemática com os inputs vindos das camadas anteriores.

Criar uma rede neural consiste geralmente em 2 etapas:
- Definir o grafo de nós e as fronteiras;
- Propagar os valores de entrada pelo grafo.

# MSE

\begin{equation*}
C(w, b) = \frac{1}{m} \sum_x [y(x) - a]^2
\end{equation*}

Aqui $w$ denota a coleta de todos os pesos na rede, $b$ todos os bias, $m$ é o número total de exemplos de treinamento, $a$ é a aproximação de $y(x)$ pela rede, tanto $a$ quanto $y(x)$ são vetores do mesmo comprimento.

# MiniFlow Example

In [53]:
import numpy as np

class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Nodes from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Nodes to which this Node passes values
        self.outbound_nodes = []
        # A calculated value
        self.value = None
        # Add this node as an outbound node on its inputs.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)

    # These will be implemented in a subclass.
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        """
        raise NotImplemented


class Input(Node):
    def __init__(self):
        # an Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Node.__init__(self)

    # NOTE: Input node is the only node that may
    # receive its value as an argument to forward().
    #
    # All other node implementations should calculate their
    # values from the value of previous nodes, using
    # self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        if value is not None:
            self.value = value


class Add(Node):
    def __init__(self, x, y):
        # You could access `x` and `y` in forward with
        # self.inbound_nodes[0] (`x`) and self.inbound_nodes[1] (`y`)
        Node.__init__(self, [x, y])

    def forward(self):
        """
        Set the value of this node (`self.value`) to the sum of its inbound_nodes.

        Your code here!
        """
        self.value = sum([node.value for node in self.inbound_nodes])

        
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        B = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + B

        
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        return 1. / (1. + np.exp(-x))
    
    def forward(self):
        h = self.inbound_nodes[0].value
        self.value = self._sigmoid(h)


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        A = self.inbound_nodes[1].value.reshape(-1, 1)
        # TODO: your code here
        y = np.concatenate([yx for yx in y])
        A = np.concatenate([ax for ax in A])
        self.value = (1 / len(y)) * sum([(yx - ax)**2 for yx, ax in zip(y, A)])

        
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(graph):
    """
    Performs a forward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()


In [54]:
import numpy as np


y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667


# Gradient Descent

In [ ]:
import random


def gradient_descent_update(x, gradx, learning_rate):
    """
    Performs a gradient descent update.
    """
    # TODO: Implement gradient descent.
    
    # Return the new value for x
    return x - learning_rate * gradx


def f(x):
    """
    Quadratic function.

    It's easy to see the minimum value of the function
    is 5 when is x=0.
    """
    return x**2 + 5


def df(x):
    """
    Derivative of `f` with respect to `x`.
    """
    return 2*x


# Random number better 0 and 10,000. Feel free to set x whatever you like.
x = random.randint(0, 10000)
# TODO: Set the learning rate
learning_rate = 0.001
epochs = 100

for i in range(epochs+1):
    cost = f(x)
    gradx = df(x)
    print("EPOCH {}: Cost = {:.3f}, x = {:.3f}".format(i, cost, gradx))
    x = gradient_descent_update(x, gradx, learning_rate)

# Backpropagation

Para cada nó, queremos alterar os valores com base no gradiente do custo em relação ao valor desse nó.

Vamos considerar uma rede com um nó linear $l_1$ , um nó sigmóide $s$, e outro nó linear $l_2$ , seguido por um nó MSE para calcular o custo, $C$.

![](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/589cc251_two-layer-graph/two-layer-graph.png)

Uma mudança em $l_2$ irá produzir uma mudança em $C$. Podemos escrever esta relação entre as mudanças como um gradiente:

\begin{equation*}
\frac{\partial C}{\partial l_2}
\end{equation*}

Isto é o que significa um gradiente, é uma inclinação, o quanto você muda o custo $\partial C$ dado uma mudança em $l_2$, $\partial l_2$. Assim podemos atribuir a cada nó uma participação no custo ou erro da rede.

Para calcular a atualização um dos pesos é preciso o gradiente do custo relacionado ao peso. Para atualizar o peso $w_2$ da imagem fica:

\begin{equation*}
\frac {\partial C}{\partial w_2}
\end{equation*}

Primeiro você tem o quanto $l_2$ afeta $C$, então quanto $w_2$ afeta $l_2$ . Multiplicando esses gradientes juntos você tem quanto $w_2$ contribui para o custo $C$.

![](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/589cd6b7_w2-backprop-graph/w2-backprop-graph.png)

Para ajudar a entender a matemática:

- https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/partial-derivatives/v/partial-derivatives-introduction
- https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/gradient-and-directional-derivatives/v/gradient
- https://www.khanacademy.org/math/ap-calculus-ab/product-quotient-chain-rules-ab/chain-rule-ab/v/chain-rule-introduction

Continuando...

Multiplicando os gratientes é a aplicação do chain rule.

\begin{equation*}
\frac {\partial C}{\partial w_2} = \frac {\partial C}{\partial l_2} \times \frac {\partial l_2}{\partial w_2}
\end{equation*}

Na formula temos que a relação entre $\partial C$ (derivada parcial do erro) e $\partial w_2$ é igual a relação entre $\partial C$ e $\partial l_2$ multiplicado pela relação entre $\partial l_2$ e $\partial w_2$. Desta forma, uma alteralão em $w_2$ vai alterar $l_2$ que por sua vez altera $C$.

Pensando como frações simples (não são frações simples exatamente) podemos anular os 2 $\partial l_2$ e ficamos com $\frac {\partial C}{\partial w_2}$.

Vamos calcular o gradiente para $l_2$.

Para lembrar, o custo ou erro $C$ é:

\begin{equation*}
C = \frac{1}{m} \sum_x [y(x) - l_2]^2
\end{equation*}

E $l_2$ é a função linear:

\begin{equation*}
l_2 = w_2 \times s + b_2
\end{equation*}

Onde $w_2$, $s$ e $b_2$ são todos vetores e $w_2 \times s$ é o "dot product" entre eles.

\begin{equation*}
\frac {\partial C}{\partial l_2} = \frac{\partial}{\partial l_2} \left[\frac{1}{m} \sum_x (y(x) - l_2)^2 \right]
\end{equation*}

\begin{equation*}
= \frac{-2}{m} \sum_x (y(x) - l_2)
\end{equation*}

\begin{equation*}
\frac {\partial l_2}{\partial w_2} = \frac{\partial}{\partial w_2} \left[w_2 \times s + b_2 \right]
\end{equation*}

\begin{equation*}
= s
\end{equation*}

E colocando tudo isso junto, temos o gradiente para $w_2$

\begin{equation*}
\frac {\partial C}{\partial w_2}= \frac{-2}{m} \sum_x (y(x) - l_2) s
\end{equation*}

Este é o gradiente usado para atualizar $w_2$. Você pode ver o que fizemos aqui, andamos de volta através do grafo e multiplicmos todos os gradientes que encontramos ao longo do caminho.

Para calcular o gradiente de $w_1$ vamos mais fundo, no mesmo método.

![](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/589cda0d_w1-backprop-graph/w1-backprop-graph.png)

Usando o chain rule, vamos escrever os gradientes para cada nó em direção ao $w_1$.

\begin{equation*}
\frac {\partial C}{\partial w_1} = \frac {\partial C}{\partial l_2} \cdot \frac {\partial l_2}{\partial s} \cdot \frac {\partial s}{\partial l_1} \cdot \frac {\partial l_1}{\partial w_1}
\end{equation*}

Então começamos a calcular o gradiente de $w_1$

\begin{equation*}
\frac {\partial l_2}{\partial s} = \frac {\partial}{\partial s} [w_2 \cdot s + b_2]
\end{equation*}

\begin{equation*}
= w_2
\end{equation*}

A próxima etapa é o gradiente da função sigmoid, $s = f(l1)$. Uma vez que estamos usando a função logística aqui, a derivada pode ser escrita em termos do próprio sigmóide.

\begin{equation*}
\frac {\partial s}{\partial l_1} = \frac {\partial}{\partial l_1} f(l_1)
\end{equation*}

\begin{equation*}
= f'(l_1)
\end{equation*}

\begin{equation*}
= f(l_1)(1 - f(l1))
\end{equation*}

\begin{equation*}
\frac {\partial l_1}{\partial w_1} = \frac {\partial}{\partial w_1} [w_1 \cdot x + b_1]
\end{equation*}

\begin{equation*}
= x
\end{equation*}

Colocando tudo isto junto:

\begin{equation*}
\frac {\partial C}{\partial w_1} = \frac {-2}{m} \sum_x (y(x) - l_2) \cdot w_2 \cdot f(l_1) \cdot (1 - f(l_1)) \cdot x
\end{equation*}

Agora podemos ver um padrão claro. Para encontrar o gradiente, basta multiplicar os gradientes para todos os nós na frente dele indo para trás a partir do custo. Esta é a idéia por trás do backpropagation. Os gradientes são passados para trás através da rede e usados com descida de gradiente para atualizar os pesos e os bias. Se um nó tiver vários nós de saída, basta somar os gradientes de cada nó.

## Implementando no MiniFlow

In [15]:
"""
Implement the backward method of the Sigmoid node.
"""
import numpy as np


class Node(object):
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1


class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        #import pdb; pdb.set_trace()
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost
            


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.

        This is the final node of the network so outbound nodes
        are not a concern.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()


In [16]:
"""
Test your network here!

No need to change this code, but feel free to tweak it
to test your network!

Make your changes to backward method of the Sigmoid class in miniflow.py
"""

import numpy as np

X, W, b = Input(), Input(), Input()
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2.], [3.]])
b_ = np.array([-3.])
y_ = np.array([1, 2])

feed_dict = {
    X: X_,
    y: y_,
    W: W_,
    b: b_,
}

graph = topological_sort(feed_dict)
forward_and_backward(graph)
# return the gradients for each Input
gradients = [t.gradients[t] for t in [X, y, W, b]]

"""
Expected output

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
"""
print(gradients)

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
